## 本代码分析的是郑州管城区的二手房挂牌信息，以此来预测某一位置二手房的价格。因为链家在一个类别下最多只能显示100页，每页默认显示30个案例，本练习主要为了预测管城回族区改善房的二手价格，价格小于140万的房子参考价值较低，所以选择了从高价往低价排序的方式进行显示，取前3000个数据作为样本。
##使用方法：运行时，首先运行main()函数，将需要的数据以分段的形式存放在本地。然后运行sumCsvfile()函数，将分段存放的数据进行汇总，形成最终的文件


In [1]:
#coding = 'utf-8'
from selenium import webdriver#导入webdriver包
from selenium.webdriver.common.by import By
import time
import csv
import re
from tqdm import tqdm

city='zz'
zone='guanchenghuizuqu'
content='ershoufang'
featureFamily='co21' 
'''
'co41'=单价从低到高
'co42'=单价从高到低
'co11'=面积从低到高
'co12'=面积从高到低
'co32'=最新发布
'''
repeatStartNum=0 #当出现bug时，程序没运行完时，修改这个参数的数值，从中断的那个批次继续开始运行

###### 生成合适的价格区间，每个价格区间保持在一定的页面数量内，链家默认显示最多100个页面，此代码默认以99作为最多页面数，如果需要可以调整

In [9]:
def getFeatureCodeNumDict(pageLimit):
    '''
    'co21bp0ep59'    总价小于等于59万
    'co21bp60ep109'  总价在60-109万之间的房子按照总价从高到低
    'co21bp110ep144' 总价在100-144万之间的房子按照总价从高到低
    'co21bp100ep145' 总价在100-145万之间的房子按照总价从高到低
    'co21bp146ep10000' 总价大于等于146万'

    '''
    startPrice=0
    priceRange=1000
    featureCodeNumDict={}

    while True:
        featureCodePriceRange= featureFamily + 'bp' + str(startPrice) + 'ep' + str(startPrice+priceRange)  
        websiteAddress = 'https://'+city+'.lianjia.com/'+content+'/'+featureCodePriceRange
        chrome= webdriver.Chrome()
        chrome.get(websiteAddress)
        totalPage= re.search(r'totalPage.+[0-9]+,',chrome.page_source)
        if totalPage == None:
            chrome.quit()
            break
        elif int(totalPage[0].split(':')[-1][:-1]) >= pageLimit:
            pageNum= int(totalPage[0].split(':')[-1][:-1])
            priceRange=int(priceRange/3*2)
        elif int(totalPage[0].split(':')[-1][:-1]) < pageLimit:
            pageNum= int(totalPage[0].split(':')[-1][:-1])
            featureCodeNumDict[featureCodePriceRange] = pageNum
            startPrice=startPrice+priceRange+1
            priceRange=1000
    return featureCodeNumDict

{'co21bp0ep38': 75, 'co21bp39ep45': 75, 'co21bp46ep52': 93, 'co21bp53ep59': 96, 'co21bp60ep64': 76, 'co21bp65ep69': 89, 'co21bp70ep74': 78, 'co21bp75ep79': 83, 'co21bp80ep84': 80, 'co21bp85ep89': 96, 'co21bp90ep94': 87, 'co21bp95ep99': 98, 'co21bp100ep104': 77, 'co21bp105ep109': 93, 'co21bp110ep114': 85, 'co21bp115ep119': 96, 'co21bp120ep124': 88, 'co21bp125ep129': 90, 'co21bp130ep134': 75, 'co21bp135ep139': 88, 'co21bp140ep144': 63, 'co21bp145ep151': 98, 'co21bp152ep158': 74, 'co21bp159ep165': 73, 'co21bp166ep176': 88, 'co21bp177ep187': 72, 'co21bp188ep204': 93, 'co21bp205ep221': 81, 'co21bp222ep247': 86, 'co21bp248ep286': 81, 'co21bp287ep374': 92, 'co21bp375ep1375': 95, 'co21bp1376ep2376': 2, 'co21bp2377ep3377': 1, 'co21bp3378ep4378': 1}


###### 获取100个列表页面的地址，每个页面包含30个房子

In [36]:
def getWebAddress(featureCode,endPageNum,startPageNum=1,city='zz',zone='guanchenghuizuqu',content='ershoufang'):
    '''city='zz' #默认城市是郑州，如选其他城市一般是首字母的缩写
       zone='guanchenghuizuqu' #默认区域是郑州的管城回族区，如选其他区域一般是全拼
       content='ershoufang' #默认搜索的类别是二手房，新房和租房的网址不太一样，不适用新房和租房
       startPageNum = 1 #待分析网站的第一个页面编号
       endPageNum = 100 #待分析网站的最后一个页面编号，安居客在一个分类下默认最多显示100个页面
       featuresCode='co22' #co22是网页地址最后的几个字符，表示了目前显示的类别，co22表示的是房子总价从高往低排，
       其他主要特征包含：
       ''=默认
       'co32'=最新发布
       'co21'=总价从低到高
       'co41'=单价从低到高
       'co42'=单价从高到低
       'co11'=面积从低到高
       'co12'=面积从高到低
    '''
    websiteLists=[] #创建空的待爬网址列表
    for i in range(startPageNum,endPageNum+1):
        websiteAddress = 'https://'+city+'.lianjia.com/'+content+'/'+zone+'/' +'pg'+str(i)+featureCode
        websiteLists.append(websiteAddress)
    return websiteLists
    print('网址获取成功')

###### 下面是获取上一步网址列表中所有房子的详细信息展示页面网址。上述100个网址中的每个都有30个房子的概要信息，概要信息中包含房屋编码，通过房屋编码可以到达每个房子的详细信息展示页面。

In [31]:
def getHouseLinks(websiteLists,sleepTime=1):
    houseLinks= [] #创建一个储存每个房子详情网址的列表
    chrome = webdriver.Chrome() #创建一个chrome浏览器，变量名字无所谓，可以随便改
    for i in websiteLists:
        chrome.get(i)
        time.sleep(sleepTime) #为了防止网页刷新太快被反爬，这里每显示一个网站默认休息一段时间
        allTitles= chrome.find_elements(By.CLASS_NAME,"title") #此处用title作为第一步筛选，因为每一个展示的房子都会有一个标题
        for title in allTitles:
            try:
                houseTitle= title.find_element(By.TAG_NAME,'a') #此处用a标签进行第二步筛选，因为每个房子的信息是存在div class=‘title’标签下的a标签里的‘
                houseLink= houseTitle.get_attribute('href') #a标签的herf属性是具体某一个房子的链接，此处是第三步筛选，把跟目标无关的信息筛出去
                if 'ershoufang' in houseLink: #经过以上几步筛选出来的内容还是不精确，这里判断一下获得的网址中是否包含'ershoufang'这个我们要查找的内容来进行进一步的筛选
                    houseLinks.append(houseLink)
                else:
                    continue
            except:
                continue
    print('房屋详情页网址获取成功')
    chrome.quit()
    return houseLinks

###### 下面这部分代码是获取每个房子的数据，每个房子用一个字典存下所有的数据，所有房子的数据最后放在一个houseDataList列表中。每个房子主要包含总价、单价、基本属性和交易属性，位置等属性，具体来说每个房子是27个属性。

In [32]:
def getHouseData(houseLinks,patch,sleepTime=0):
    houseDataList = [] #创建一个存放每一个房子数据的列表，每个房子的信息以一个字典的形式存放
    chrome= webdriver.Chrome()
    houseLinksBar = tqdm(houseLinks,ncols=70) #这里使用tqdm库创建一个进度条，将每个patch需要循环获取的300个数据放入其中生成一个houseLinksBar。
    for houseLink in houseLinksBar: #这个houseLinkBar跟houseLinks基本是一样的，但是在进入循环之后会多一个根据当前处理的数据显示进度的功能
        houseLinksBar.set_description(f'处理第{patch}批次中，共10批次')
        dictTmp={}
        chrome.get(houseLink)
        time.sleep(sleepTime) #获取网站后休息一定时间再获取下一条，防止被ban
        #获取第一个key-value：链家编号
        houseRecord = chrome.find_element(By.CLASS_NAME,'houseRecord')
        labelHouseCode=houseRecord.find_element(By.CLASS_NAME,'label')
        houseCode=houseRecord.find_element(By.CLASS_NAME,'info')
        dictTmp= {labelHouseCode.text:houseCode.text[0:12]}
        #获取房子在网站上的标题内容
        titleTmp= r'class=\"main\" title=\".+\"'
        title= re.search(titleTmp,chrome.page_source)[0]
        dictTmp['备注'] = title.split('=')[-1][1:-1]
        #获取第二个key-value:总价
        totalPrice = chrome.find_element(By.CLASS_NAME,'total')
        dictTmp['房屋总价'] = totalPrice.text
        #获取第三个key-value:单价
        unitPriceValue = chrome.find_element(By.CLASS_NAME,'unitPriceValue')
        dictTmp['房屋单价'] = unitPriceValue.text
        #获取房子剩下的的基本属性，第4到23个值
        otherInfoListTmp = chrome.find_element(By.CLASS_NAME,'introContent')
        otherInfoList = otherInfoListTmp.find_elements(By.TAG_NAME,'li')
        for i in otherInfoList:
            dictTmp[i.text[0:4]]=i.text[4:] #前四个字符是key，后面的字符是value
        #获取房子所在的小区编号
        resblockIdTmp= r'resblockId:.+\''
        resblockId= re.search(resblockIdTmp,chrome.page_source)[0]
        dictTmp['小区编号'] = resblockId.split(':')[1][1:-1]
        #获取房子所在的小区名称
        resblockNameTmp= r'resblockName:.+\''
        resblockName= re.search(resblockNameTmp,chrome.page_source)[0]
        dictTmp['小区名称'] = resblockName.split(':')[1][1:-1]
        #获取房子所在的小区经纬度
        resblockPositionTmp= r'resblockPosition:.+\''
        resblockPosition= re.search(resblockPositionTmp,chrome.page_source)[0]
        dictTmp['小区经度'] = resblockPosition.split('\'')[1].split(',')[0]
        dictTmp['小区维度'] = resblockPosition.split('\'')[1].split(',')[1]
        #获取房子的配套信息
        
        #一个房子的信息收集完毕，把字典数据写入列表中
        houseDataList.append(dictTmp)
    chrome.quit()
    return houseDataList

###### 将获得的数据写入csv文件，进行本地保存

In [33]:
def saveHouseData(houseDataList,fileName,patch):
    #不同房屋所包含的属性数量可能不同，此处取前三个房子的属性，比较属性数量最多的一个作为整个表格的header
    houseDataHeader = []
    for i in houseDataList[0:3]:
        if len(list(i.keys()))>len(houseDataHeader):houseDataHeader=list(i.keys())
    
    with open(fileName,'w',newline='',encoding= 'utf-8') as csvfile:
        '''
        restval='暂无数据'表示当出现有一个房子存在某一项信息缺失时，会在这个房子的这一项数据填上暂无数据
        extrasaction='ignore'表示当有房子存在比header更多的属性时，这些多余的属性会被丢掉，如果把ignore换成raise，会抛出错误
        '''
        writer = csv.DictWriter(csvfile,fieldnames= houseDataHeader,restval='暂无数据',extrasaction='ignore') 
        writer.writeheader()
        for i in houseDataList:
            writer.writerow(i)
    print(f'第{patch}个数据已写入完毕，还剩{9-patch}个数据待写入')
    return houseDataHeader

In [29]:
def main(dividedIntoPart=10,repeatStartNum=0):

    '''为了防止大量数据一次性写入错误导致反复运行等待，此处将所有数据分成多个批次进行获取和写入，
    默认分为10个批次进行处理，每批次的结果分别存成一个文件，后面再对csv文件进行合并。
    houseLinks是所有待分析房子的链接，必须传入
    dividedIntoPart是将所有数据分成多少个部分分别存放在csv中，默认为10个
    repeatStartNum是如果出现错误，从第几个文件重新开始，默认是0，既从第一个开始
    '''
    featureCodeNumDict = getFeatureCodeNumDict(100) 
    for i,featureCode in enumerate(list(featureCodeNumDict.keys())):
        print(f'整个数据列表总共{len(list(featureCodeNumDict.keys()))}部分，开始第{i}部分')
        endPageNum= featureCodeNumDict[featureCode]
        websiteLists = getWebAddress(featureCode=featureCode,endPageNum=endPageNum,startPageNum=1,city=city,zone=zone,content=content,featureCode=featureCode) #函数名称也是一个变量，并且是个全局变量，所以这里不需要通过参数传递将函数名称从外面传递到主函数里面
        houseLinks = getHouseLinks(websiteLists)
        csvfileNameList=[] #创建一个存储生成的csv文件名称的列表，用于合并csv文件
        for patch in range(repeatStartNum,dividedIntoPart): 
            '''
            #这里尝试用递归解决一个问题，网页有时候响应超出时间，会导致程序错误，这里通过一个try&except和递归来处理。
            在每一个分段里，首先会运行try/if的内容，如果没有问题，会进行下一个分段。如果出现问题，则触发excep并进一
            步触发main()函数，但是main()的repeatStartNum参数将被修改为patch，也就是将从失败的这个patch值重新开始迭代。
            当所有的数据都被分析完成之后，递归将会将之前没有执行完的循环继续执行，为了防止重复分析，会先判断之前有没有
            相关数据，如果有则直接continue。
            '''
            csvfileName=city+zone.title()+content.title()+featureCode.title()+str(patch)+'.csv'
    # try: 
            if csvfileName in csvfileNameList:
                continue
            else:
                if patch != dividedIntoPart-1:
                    houseDataList = getHouseData(houseLinks[int(len(houseLinks)/dividedIntoPart*patch):int(len(houseLinks)/dividedIntoPart*(patch+1))],patch)
                else:
                    houseDataList = getHouseData(houseLinks[int(len(houseLinks)/dividedIntoPart*patch):],patch)
                houseDataHeader = saveHouseData(houseDataList,csvfileName,patch)
                csvfileNameList.append(csvfileName)
    # except TimeoutException: 
    #     main(dividedIntoPart=10,repeatStartNum=patch)
        return houseDataHeader

In [ ]:
if __name__=='__main__':
    main()